In [1]:
import pandas as pd
from elftools.elf.elffile import ELFFile

In [2]:
csv_name = "~/faults_eddi_inter-rasm.csv"
bin_name = "../out_eddi_inter-rasm_1.elf"

data = pd.read_csv(csv_name)
data

,attempt,stop_addr,stop_fn,delay,target,bitflip,code,seed
0,0,0x08003a16,xTaskIncrementTick,0.491,0x200095b3,4,0,3735928559
1,1,0x08005f2e,prvIdleTask,0.543,0x200095ce,128,0,3735928559
2,2,0x0800dc3e,xTaskResumeAll_ret_dup,0.452,0x2000b1da,16,0,3735928559
3,3,0x0804421a,Reset_Handler,0.021,0x20007c11,2,0,3735928559
4,4,0x080001b6,memset,0.750,0x20005cdc,8,0,3735928559
...,...,...,...,...,...,...,...,...
75830,75830,0x08001eac,memcpy_to_duplicate,0.422,0x2000653f,128,0,3735928559
75831,75831,0x0800a610,xTaskGetSchedulerState_ret_dup,0.965,0x20003f7a,8,0,3735928559
75832,75832,0x08005c10,vTaskSwitchContext,0.489,0x2000561e,2,0,3735928559
75833,75833,0x080258ba,RandomInteger_ret_dup,0.709,0x20007dea,128,0,3735928559


In [3]:
symtable = dict()

with open(bin_name, 'rb') as elffile:
    for section in ELFFile(elffile).iter_sections():
        try:
            for sym in section.iter_symbols():
                if (sym.name != "ucHeap" and sym.entry['st_info']['type'] == 'STT_OBJECT'):
                    symtable[sym.name] = {"addr": sym.entry['st_value'], "size": sym.entry['st_size'] * 8}
        except: 
            continue

In [4]:
data_size = data.shape[0]

new_data = [None for i in range(data_size)]

for idx, row in data.iterrows():
    for sym in symtable.keys():
        try:
            if int(row['target'][2:], base=16) in range(symtable[sym]['addr'], symtable[sym]['addr']+symtable[sym]['size'], 1):
                #print(symtable[sym])
                new_data[idx] = sym
                break
        except:
            break
    print(str(idx) + "/" + str(data_size), end = "\r")

data['struct'] = new_data

In [5]:
data.to_csv("faults_eddi_inter-rasm_names.csv")